# Importer disse bibliotekene for at programmet skal fungere

In [5]:
import fiona
import os
import geopandas as gpd
import folium
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
import pyogrio
from shapely.geometry import Point

## Kjør denne cellen for at funksjoner for konvertering skal iverksettes

In [ ]:
def convert_ar50_to_fgb(input_file, output_file):
    """
    Konverterer AR50-data fra GML eller GDB til FlatGeobuf (FGB),
    og filtrerer kun ut polygon-geometrier.
    """
    file_ext = os.path.splitext(input_file)[1].lower()

    # Bruk riktig lag for hver filtype
    if file_ext == ".gml":
        layer_name = "ArealressursFlate"
        gdf = gpd.read_file(input_file, layer=layer_name)

    elif file_ext == ".gdb":
        layer_name = "ar50"
        if layer_name not in fiona.listlayers(input_file):
            raise ValueError(f"Laget '{layer_name}' finnes ikke i GDB-filen.")
        gdf = gpd.read_file(input_file, layer=layer_name)

    else:
        raise ValueError(f"Ukjent eller ikke støttet filtype: {file_ext}")

    # Filtrer kun polygoner
    gdf = gdf[gdf.geom_type.isin(['Polygon', 'MultiPolygon'])]

    # Lagre som FlatGeobuf
    gdf.to_file(output_file, driver="FlatGeobuf")
    print(f"Lagret '{layer_name}' som {output_file}")

## Start konvertering av GML og GDB filer til FGB

In [ ]:
input_gml = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/42_25832_ar50_gml.gml"
input_gdb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/42_25832_ar50_gdb.gdb"

# Konverter GML til FGB
output_fgb_gml = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gml.fgb"
convert_ar50_to_fgb(input_gml, output_fgb_gml)

# Konverter GDB til FGB
output_fgb_gdb = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gdb.fgb"
convert_ar50_to_fgb(input_gdb, output_fgb_gdb)

## Her kan du legge inn din ønskede kolonne og verdi du vil filterer etter! Du kan også filterer etter hvilken postkode du vil filtrere etter og hvor mange rader du øsnker skal vises.

In [16]:
##############################################
# Funksjon for bufferfiltrering
##############################################
def filtering_table_map_buffer_ar50(fgb_path, column, center_x, center_y, buffer_radius):
    """
    Lager et kart basert på AR50-data med partial read: Leser kun nødvendig kolonne + geometri.
    Viser alle objekter innenfor bufferområdet med fargekoding etter verdien i 'column'.
    Objekter som delvis ligger innenfor bufferen, klippes slik at bare den delen som ligger
    innenfor bufferen vises. Rader med verdien 98 i den angitte kolonnen ekskluderes.

    Returnerer en tuple med:
      - folium Map, og
      - GeoDataFrame med de filtrerte dataene.
    """
    # Les kun nødvendig kolonne + geometri med pyogrio
    gdf = pyogrio.read_dataframe(fgb_path, columns=["geometry", column])

    # Projiser til UTM for metrisk beregning (EPSG:25832)
    gdf = gdf.to_crs(epsg=25832)

    # Lag senterpunkt og buffer
    center_point = Point(center_x, center_y)
    center_point = gpd.GeoSeries([center_point], crs="EPSG:4326").to_crs(epsg=25832).iloc[0]
    buffer_geom = center_point.buffer(buffer_radius)
    buffer_gdf = gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:25832").to_crs(epsg=4326)

    # Klipp ut objekter slik at vi får med den delen som ligger innenfor bufferen
    gdf_clipped = gpd.clip(gdf, buffer_geom)

    # Ekskluder rader der 'column' har verdien 98
    gdf_clipped = gdf_clipped[gdf_clipped[column] != 98]

    # Konverter til EPSG:4326 for visning
    table_data = gdf_clipped.to_crs(epsg=4326)

    # Bygg et fargekart basert på unike verdier i 'column'
    unique_values = table_data[column].unique().tolist()
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'darkred', 'lightred', 'beige',
              'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink',
              'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
    color_map = {val: colors[i % len(colors)] for i, val in enumerate(unique_values)}
    print("Fargekart:", color_map)

    # Beregn kartets senter (bruker bufferen)
    center_coords = [buffer_gdf.geometry.unary_union.centroid.y, buffer_gdf.geometry.unary_union.centroid.x]
    m = folium.Map(location=center_coords, zoom_start=12, tiles="cartodb positron")

    # Legg til bufferområdet som et lag
    folium.GeoJson(
        buffer_gdf,
        name="Bufferområde",
        style_function=lambda x: {"color": "black", "weight": 1, "fillOpacity": 0.0},
    ).add_to(m)

    # Lag et lag for hver unik verdi i 'column'
    for val in unique_values:
        subset = table_data[table_data[column] == val]
        layer = folium.GeoJson(
            subset,
            name=f"{column}: {val}",
            style_function=lambda feature, val=val: {
                "color": color_map.get(val, "black"),
                "weight": 1,
                "fillOpacity": 0.4
            },
            tooltip=folium.GeoJsonTooltip(fields=[column], aliases=[f"{column}:"])
        )
        layer.add_to(m)

    folium.LayerControl().add_to(m)

    return m, table_data

##############################################
# Funksjon for standard filtrering (uten buffer)
##############################################
def filter_table_map_Komid_ar50(ar50_path, filter_column, filter_value, komid_value=None, n_rows=10):
    """
    Leser AR50-data og filtrerer ut de radene der:
      - Dersom filter_value ikke er tom, filtreres det slik at verdien i filter_column er lik filter_value.
        (Dersom filter_value er tom, benyttes alle radene i kolonnen.)
      - Verdien 98 i filter_column ekskluderes alltid.
      - Om komid_value ikke er None (eller 0), filtreres det også på at kolonnen "komid" er lik komid_value.

    Returnerer to verdier:
      1. De første n_rows radene (som en GeoDataFrame) med kolonnene:
         - filter_column, "rid", "komid", "geometry"
      2. Det totale antallet rader som matcher filterbetingelsene.
    """
    gdf = gpd.read_file(ar50_path)
    # Ekskluder rader med verdien 98 i filter_column
    if filter_column in gdf.columns:
        gdf = gdf[gdf[filter_column] != 98]
    # Filtrer med filter_value (om angitt)
    if filter_value is None or filter_value == "":
        gdf_filtered = gdf.copy()
    else:
        gdf_filtered = gdf[gdf[filter_column] == filter_value]
    # Filtrer på komid hvis angitt
    if komid_value is not None and komid_value != 0:
        gdf_filtered = gdf_filtered[gdf_filtered["komid"] == komid_value]

    total_count = len(gdf_filtered)
    desired_cols = [filter_column, "rid", "komid", "geometry"]
    desired_cols = [col for col in desired_cols if col in gdf_filtered.columns]
    return gdf_filtered[desired_cols].head(n_rows), total_count

##############################################
# Del 1: Filvalg
##############################################
file_label = widgets.Label(value="Filbane:")
file_path_widget = widgets.Text(
    value="/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gdb.fgb",
    layout=widgets.Layout(width='500px')
)
file_done_button = widgets.Button(
    description="Ferdig",
    button_style='info',
    layout=widgets.Layout(width='150px')
)
file_output = widgets.Output()
file_box = widgets.VBox([file_label, file_path_widget, file_done_button])

##############################################
# Del 2: Filtreringsparametre
##############################################
# Standard filterparametre
filter_col_label = widgets.Label(value="Filterkolonne:")
filter_column_widget = widgets.Text(value="artype", layout=widgets.Layout(width='500px'))

filter_val_label = widgets.Label(value="Filterverdi (la stå tom for åvise alle verdier):")
filter_value_widget = widgets.Text(value="10", layout=widgets.Layout(width='500px'))

komid_val_label = widgets.Label(value="Komid (Skriv 0 for å ignorere):")
komid_value_widget = widgets.Text(value="4204", layout=widgets.Layout(width='500px'))

# Nye bufferparametre
buffer_rad_label = widgets.Label(value="Bufferradius (m): (Skriv 0 for å ikke bruke bufferen)")
buffer_radius_widget = widgets.IntText(value=0, layout=widgets.Layout(width='500px'))

center_x_label = widgets.Label(value="Buffer senter X (lon):")
center_x_widget = widgets.Text(value="7.9956", layout=widgets.Layout(width='500px'))

center_y_label = widgets.Label(value="Buffer senter Y (lat):")
center_y_widget = widgets.Text(value="58.1467", layout=widgets.Layout(width='500px'))

n_rows_label = widgets.Label(value="Antall rader:")
n_rows_widget = widgets.IntText(value=10, layout=widgets.Layout(width='500px'))

run_filter_button = widgets.Button(
    description="Kjør filtrering",
    button_style='info',
    layout=widgets.Layout(width='150px')
)
filter_output = widgets.Output()

# Alle filtrerings-widgetene pakkes inn; denne boksen skjules til filvalg er bekreftet.
filter_widgets_box = widgets.VBox([
    filter_col_label, filter_column_widget,
    filter_val_label, filter_value_widget,
    komid_val_label, komid_value_widget,
    buffer_rad_label, buffer_radius_widget,
    center_x_label, center_x_widget,
    center_y_label, center_y_widget,
    n_rows_label, n_rows_widget,
    run_filter_button,
    filter_output
])
filter_widgets_box.layout.display = 'none'

##############################################
# Hendelse for filvalg ("Ferdig")
##############################################
def on_file_done_clicked(b):
    with file_output:
        file_output.clear_output()
        file_path = file_path_widget.value.strip()
        if not file_path:
            print("Feil: Du må oppgi en gyldig filbane.")
            return
        try:
            sample_gdf = gpd.read_file(file_path).head(100)
            overview_dict = {}
            excluded_cols = ["geometry", "arkartstd", "kilde"]
            for col in sample_gdf.columns:
                if col in excluded_cols:
                    continue
                unique_vals = sample_gdf[col].unique()
                overview_dict[col] = ", ".join(str(val) for val in unique_vals[:5])
            overview_df = pd.DataFrame(list(overview_dict.items()), columns=["Kolonne", "Eksempelverdier"])
            print("Her er hvilke kolonner og verdier du kan filtrere mellom!")
            display(overview_df)
            filter_widgets_box.layout.display = ''
        except Exception as e:
            print("Kunne ikke lese filen. Feilmelding:", e)

file_done_button.on_click(on_file_done_clicked)

##############################################
# Hendelse for filtreringsknappen ("Kjør filtrering")
##############################################
def on_run_filter_clicked(b):
    with filter_output:
        filter_output.clear_output()
        try:
            ar50_path = file_path_widget.value.strip()
            filter_column = filter_column_widget.value.strip()
            filter_value_str = filter_value_widget.value.strip()
            komid_value_str = komid_value_widget.value.strip()
            n_rows = n_rows_widget.value

            # Håndter filter_value-konvertering; hvis tom, settes den til ""
            if filter_value_str == "":
                filter_value = ""
            else:
                try:
                    if "." in filter_value_str:
                        filter_value = float(filter_value_str)
                    else:
                        filter_value = int(filter_value_str)
                except ValueError:
                    filter_value = filter_value_str

            # Håndter komid_value
            if komid_value_str.lower() in ["", "none", "0"]:
                komid_value = None
            else:
                try:
                    if "." in komid_value_str:
                        komid_value = float(komid_value_str)
                    else:
                        komid_value = int(komid_value_str)
                except ValueError:
                    komid_value = komid_value_str

            # Sjekk om buffer skal brukes: Dersom bufferradius > 0 og senterkoordinater er fylt ut
            buffer_radius = buffer_radius_widget.value
            center_x_val = center_x_widget.value.strip()
            center_y_val = center_y_widget.value.strip()

            if buffer_radius and buffer_radius > 0 and center_x_val != "" and center_y_val != "":
                try:
                    center_x = float(center_x_val)
                    center_y = float(center_y_val)
                except ValueError:
                    print("Feil: Senterkoordinatene må være gyldige tall (desimaltall tillatt).")
                    return
                # Bruk bufferfiltrering – her ignoreres filterverdi og komid
                m, table_data = filtering_table_map_buffer_ar50(
                    ar50_path, filter_column, center_x, center_y, buffer_radius
                )
                total_count = len(table_data)
                # Eventuelt kan du vise bare de første n_rows radene
                result_table = table_data.head(n_rows)
            else:
                # Bruk standard filtrering med filterverdi og evt. komid
                result_table, total_count = filter_table_map_Komid_ar50(
                    ar50_path,
                    filter_column=filter_column,
                    filter_value=filter_value,
                    komid_value=komid_value,
                    n_rows=n_rows
                )
                # Konverter CRS til EPSG:4326 for folium-visning (om nødvendig)
                if result_table.crs and result_table.crs != "EPSG:4326":
                    result_table = result_table.to_crs("EPSG:4326")
                # Lag et kart sentrert på rader som hentes (beregn centroid for forent geometri)
                center = result_table.geometry.union_all().centroid
                map_center = [center.y, center.x]
                m = folium.Map(location=map_center, zoom_start=12)
                folium.GeoJson(result_table).add_to(m)
                folium.LayerControl().add_to(m)

            if result_table.empty:
                print("Ingen data funnet med de gitte filterparameterne.")
                return

            print("Filtrerte data:")
            print(f"Hentet totalt {total_count} rader, viser {len(result_table)} rader.")
            display(result_table)
            print("Interaktivt kart:")
            display(m)
        except Exception as e:
            print("En feil oppstod under filtreringen:", e)

run_filter_button.on_click(on_run_filter_clicked)

##############################################
# Vis sammensatt grensesnitt
##############################################
file_output_box = widgets.VBox([file_output])
display(widgets.VBox([file_box, file_output_box, filter_widgets_box]))
